In [140]:
#!pip install implicit==0.4.4

In [141]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

# Функции из 1-ого вебинара
import os, sys
from metrics import precision_at_k, recall_at_k

In [142]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [143]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [144]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [145]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [146]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

Wall time: 7.85 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[16056902, 12263730, 879003, 9854664, 6391698]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[948406, 99172, 837073, 872098, 6772706]"


In [147]:
#Input
#получили значение sales_value для каждого item_id
weight = data_train.groupby('item_id')['sales_value'].sum().reset_index()
#print(weight)

In [148]:
## Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
weights = (weight['sales_value']/sum(weight['sales_value']))
#print(weights)
sum(weights)

0.9999999999995316

In [149]:
items_weights_list = list(zip(items, weights))

#items_weights: pd.DataFrame
items_weights = pd.DataFrame(items_weights_list, columns = ['item_id', 'weight'])

In [150]:
def weighted_random_recommendation(items_weights, n=5):

    items=(items_weights['item_id'])
    weights=(items_weights['weight'])
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    recs = np.random.choice(items , p=weights, size=n, replace=False)
    #print(recs)
    
    return recs.tolist()

In [151]:
%%time
weighted_random_recommendation(items_weights,n=5)
# your_code

Wall time: 18 ms


[1007085, 881893, 13416735, 849003, 1049296]

In [152]:
%%time

#items = data_train.item_id.unique()

result['weighted_random_rec'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

Wall time: 3.9 s


,user_id,actual,random_recommendation,weighted_random_rec
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[16056902, 12263730, 879003, 9854664, 6391698]","[980278, 1105059, 1131490, 7441718, 5819183]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[948406, 99172, 837073, 872098, 6772706]","[927479, 6443362, 932962, 851447, 13879509]"


In [153]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [154]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 142 ms


,user_id,actual,random_recommendation,weighted_random_rec,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[16056902, 12263730, 879003, 9854664, 6391698]","[980278, 1105059, 1131490, 7441718, 5819183]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[948406, 99172, 837073, 872098, 6772706]","[927479, 6443362, 932962, 851447, 13879509]","[6534178, 6533889, 1029743, 6534166, 1082185]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

#### Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров

In [155]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [156]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [157]:
data_train_5000 =data_train[data_train['item_id'].isin(top_5000)]

In [158]:
%%time

items = data_train_5000.item_id.unique()

result['random_rec_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

Wall time: 895 ms


,user_id,actual,random_recommendation,weighted_random_rec,popular_recommendation,random_rec_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[16056902, 12263730, 879003, 9854664, 6391698]","[980278, 1105059, 1131490, 7441718, 5819183]","[6534178, 6533889, 1029743, 6534166, 1082185]","[828891, 1112825, 7463287, 988520, 1079987]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[948406, 99172, 837073, 872098, 6772706]","[927479, 6443362, 932962, 851447, 13879509]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1036347, 1111722, 889692, 1111072, 904435]"


In [159]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0005:random_recommendation
0.0006:weighted_random_rec
0.1552:popular_recommendation
0.0055:random_rec_5000


#### Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей  𝐾 .

In [160]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix)

In [161]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [162]:
%%time

def ItemItemRec(result, K=4, num_threads=4):

    model = ItemItemRecommender(K, num_threads) # K - кол-во билжайших соседей
    #print(f'model-{model}')


    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)


    recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                            user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                            N=5, # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)
    #print(f'recs-{recs}')
    
    result[f'itemitem_{K}'] = result['user_id'].apply(lambda user_id: [id_to_itemid[rec[0]]  
                                                              for rec in model.recommend(userid=userid_to_id[user_id], 
                                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                                N=5, 
                                                                filter_already_liked_items=False, 
                                                                filter_items=None, 
                                                                recalculate_user=True)
                                                                             ])
    #print(f'result-{result}')
    return result

Wall time: 997 µs


In [163]:
k_list = [k for k in range(1, 8)]

for K in k_list:
    ItemItemRec(result, K, num_threads=6)
    

In [164]:
result.head(2)

,user_id,actual,random_recommendation,weighted_random_rec,popular_recommendation,random_rec_5000,itemitem_1,itemitem_2,itemitem_3,itemitem_4,itemitem_5,itemitem_6,itemitem_7
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[16056902, 12263730, 879003, 9854664, 6391698]","[980278, 1105059, 1131490, 7441718, 5819183]","[6534178, 6533889, 1029743, 6534166, 1082185]","[828891, 1112825, 7463287, 988520, 1079987]","[1082185, 995242, 1029743, 840361, 904360]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 1127831, 995242, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[948406, 99172, 837073, 872098, 6772706]","[927479, 6443362, 932962, 851447, 13879509]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1036347, 1111722, 889692, 1111072, 904435]","[1082185, 1098066, 6534178, 826249, 1127831]","[1082185, 981760, 1098066, 1127831, 826249]","[1082185, 981760, 1098066, 826249, 1127831]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]"


In [165]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0005:random_recommendation
0.0006:weighted_random_rec
0.1552:popular_recommendation
0.0055:random_rec_5000
0.2195:itemitem_1
0.2063:itemitem_2
0.17:itemitem_3
0.151:itemitem_4
0.1541:itemitem_5
0.1591:itemitem_6
0.1642:itemitem_7
